In [1]:
import warnings
warnings.filterwarnings("ignore")  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from federated_utils_fedavg_copy import *

 
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [3]:
#declear path to your data
drebin_data_path = r'data\drebin.csv'
malgenome_data_path = r'data\malgenome.csv'
kronodroid_data_path = r'data\kronodroid.csv'
TUANDROMD_data_path=r'data\TUANDROMD.csv'



Drebin_data = pd.read_csv(drebin_data_path, header = None)

Malgenome_data = pd.read_csv(malgenome_data_path)

Tuandromd_data=pd.read_csv(TUANDROMD_data_path)

kronodroid_data=pd.read_csv(kronodroid_data_path)
Kronodroid_data = kronodroid_data.iloc[:,range(1,kronodroid_data.shape[1])]




In [4]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from itertools import chain
import copy

def train_model_prox(model,global_model, train_loader, loss_fn, optimizer,client ,rho,Z_weights,x_hats,mu=0.01):
    model.train()
    for i in range(2):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)

            # Add the proximal term
            # for param, param_global in zip(model.parameters(), global_model.parameters()):
            #     loss += (mu / 2) * torch.norm(param - param_global, p=2)**2


            for z_weight, (param, param_global) in zip(Z_weights[int(client[-1])-1], zip(model.parameters(), global_model.parameters())):
                loss += (mu / 2) * torch.norm(param - param_global, p=2)**2 + torch.sum(z_weight * (param - param_global))

            # Now 'inner_product' contains the inner product between model.parameters() and global_model.parameters()

            loss.backward(retain_graph=True)  # Set retain_graph=True to retain the computation graph
            optimizer.step()



def update_z_parameter(local_model,global_model,Z_weights,x_hats,client,rho):
    # Update Z_weight
    for i, (z_weight, (param, param_global)) in enumerate(zip(Z_weights[int(client[-1])-1], zip(local_model.parameters(), global_model.parameters()))):
        Z_weights[int(client[-1])-1][i] += rho * (param - param_global)
    temp = copy.copy(x_hats[int(client[-1])-1])
    # temp2 = copy.copy(x_hats[int(client[-1])-1])

    # Update x_hats
    # for x_hat, z_weight, param_model in zip(x_hats[int(client[-1])-1],(Z_weights[int(client[-1])-1], local_model.parameters()) ):
    #     x_hat = param_model +  z_weight/rho
    for i, (x_hat, z_weight, param_model) in enumerate(zip(x_hats[int(client[-1])-1], Z_weights[int(client[-1])-1], local_model.parameters())):
        x_hats[int(client[-1])-1][i] = param_model + z_weight / rho
    # print(x_hats[int(client[-1])-1])
    # print(delta_x_hats[int(client[-1])-1])


    return Z_weights,x_hats



# def update_avg_with_delta(avg_grad, delta_x_hats):
#     '''Update avg_grad by adding the corresponding elements from delta_x_hats'''
#     for avg_layer, delta_layers in zip(avg_grad, zip(*delta_x_hats)):
#         avg_layer += delta_layers
#     return avg_grad







all_avg = []
all_std = []
mu = 0.01
rho =0.01
n_clients = [10]
n_round = [100]

dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd']
for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d == 1:
        use_data = Malgenome_data
    elif d == 2:
        use_data = Kronodroid_data
    elif d == 3:
        use_data = Tuandromd_data

    print('===================================================================================================')
    print('Working with:', dataset[d])
    print('===================================================================================================')

    for r in n_round:  # number of rounds loop
        comms_round = r
        for cl in n_clients:  # number of clients loop
            number_of_clients = cl

            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')

            features = np.array(use_data.iloc[:, range(0, use_data.shape[1] - 1)])  # feature set
            labels = use_data.iloc[:, -1]  # labels --> B : Benign and S

            # Do feature scaling
            X = preprocessing.StandardScaler().fit(features).transform(features)

            # binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)

            # split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y, shuffle=True,
                                                                test_size=0.2,
                                                                random_state=100)

            # create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            # process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)

            # process and batch the test set
            test_batched = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                                                                     torch.tensor(y_test, dtype=torch.float32)),
                                                       batch_size=len(y_test), shuffle=False)

            # ==============================================
            # Traditional FedAvg 2017
            # ==============================================
            # -----------------------------------------------

            all_results = list()

            # create optimizer
            lr = 0.01
            loss = nn.BCELoss()
            optimizer = optim.SGD

            # initialize global model
            smlp_global = SimpleMLP(X.shape[1], 1)
            global_model = smlp_global
            # -----------------------------------------------
            Z_weights=[]
            x_hats = []
            for i in range(number_of_clients):
                Z_weight = [torch.zeros_like(param) for param in global_model.parameters()]
                Z_weights.append(Z_weight)
                x_hat = [torch.zeros_like(param) for param in global_model.parameters()]
                x_hats.append(x_hat)
            print('|=======================|')
            print('|Traditional FedADMM 2022|')
            print('|=======================|')

            # commence global training loop
            for comm_round in range(comms_round):
                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = [param.data.clone() for param in global_model.parameters()]

                # initial list to collect local model weights after scaling
                scaled_local_weight_list = list()
                scaled_x_hats_list = list()

                # randomize client data - using keys
                client_names = list(clients_batched.keys())
                random.shuffle(client_names)


                for client in client_names:
                    smlp_local = SimpleMLP(X.shape[1], 1)
                    local_model = smlp_local
                    # set local model weight to the weight of the global model
                    local_model.load_state_dict({name: param.clone() for name, param in zip(local_model.state_dict(), global_weights)})
                    optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)

                    # fit local model with client's data
                    train_loader = DataLoader(TensorDataset(torch.tensor(clients_batched[client].dataset.tensors[0], dtype=torch.float32),
                                                            torch.tensor(clients_batched[client].dataset.tensors[1], dtype=torch.float32)),
                                              batch_size=32, shuffle=True)

                    train_model_prox(local_model, global_model,train_loader, loss, optimizer,client,rho,Z_weights,x_hats,mu)
                    Z_weights,x_hats = update_z_parameter(local_model,global_model,Z_weights,x_hats,client,rho)
   

                    # scale the model weights and add to the list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    # scaled_weights = scale_model_weights(local_model.state_dict().values(), scaling_factor)
                    scaled_x_hats = scale_model_weights(x_hats[int(client[-1])-1], scaling_factor)
                    # print(local_model.state_dict().values())
                    scaled_x_hats_list.append(scaled_x_hats)
                    # scaled_local_weight_list.append(scaled_weights)

                    # clear session to free memory after each communication round
                    torch.cuda.empty_cache()
                
                # ..
  
                average_weights = sum_scaled_weights(scaled_x_hats_list)
                # print(len(average_weights), "asdsa")

                # update global model
                for param, avg_param in zip(global_model.parameters(), average_weights):
                    param.data.copy_(avg_param)

                # test global model and print out metrics after each communications round
                for X_test_batch, Y_test_batch in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test_batch, Y_test_batch, global_model, comm_round)
                    all_results.append([global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr])

            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = f'results/round-{r}/{cl}-clients/FedADMM-{dataset[d]}-results.csv'
            all_R.to_csv(flname, index=None)

            all_avg.append(np.concatenate(([dataset[d], r, cl], np.mean(all_results, axis=0))))  # Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d], r, cl], np.std(all_results, axis=0))))  # Storing std values for each dataset

ALL_AVG = pd.DataFrame(all_avg, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedADMM-results.csv')

ALL_STD = pd.DataFrame(all_std, columns=['Dataset', 'num of round', 'num of clients', 'global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedADMM-all-std-results.csv')


Working with: Drebin
---------------------------------------------
No. of Clients: 5
No. of Rounds: 200
---------------------------------------------
|=======================|
|Traditional FedADMM 2022|
|=======================|
comm_round: 0 | global_acc: 63.132% | global_loss: 0.6382462382316589 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.8215818586119314| flobal_FPR: 1.0 
comm_round: 1 | global_acc: 63.132% | global_loss: 0.5355787873268127 | global_f1: 0.0 | global_precision: 1.0 | global_recall: 0.0 | global_auc: 0.924402098584467| flobal_FPR: 1.0 
comm_round: 2 | global_acc: 94.747% | global_loss: 0.29206186532974243 | global_f1: 0.9283121597096189 | global_precision: 0.9342465753424658 | global_recall: 0.9224526600541028 | global_auc: 0.9802591748967588| flobal_FPR: 0.0775473399458972 
comm_round: 3 | global_acc: 94.481% | global_loss: 0.15951257944107056 | global_f1: 0.9256272401433693 | global_precision: 0.9198575244879786 | global_recall: 0.9

KeyboardInterrupt: 

In [ ]:
print((all_avg[0]))

In [ ]:

all_avg =[]

all_std =[]

n_clients = [5,10,15]
n_round = [10,20]


dataset = ['Drebin', 'Malgenome', 'Kronodroid', 'Tuandromd' ]


for d in range(0,1):
    if d == 0:
        use_data = Drebin_data
    elif d==1:
        use_data = Malgenome_data
    elif d==2:
        use_data = Kronodroid_data
    elif d==3:
        use_data = Tuandromd_data
        
        
    print('===================================================================================================')
    print('Working with:',dataset[d])
    print('===================================================================================================')

    for r in n_round: #number of rounds loop
        comms_round = r
        for cl in n_clients: #number of clients loop
            number_of_clients = cl

            # from sklearn.utils import shuffle
            # use_data = shuffle(use_data)
            # use_data
            print('---------------------------------------------')
            print('No. of Clients:', number_of_clients)
            print('No. of Rounds:', comms_round)
            print('---------------------------------------------')


            features = np.array(use_data.iloc[:,range(0,use_data.shape[1]-1)]) #feature set

            labels = use_data.iloc[:,-1] #labels --> B : Benign and S


            #Do feature scaling 


            X = preprocessing.StandardScaler().fit(features).transform(features)


            #binarize the labels
            lb = LabelBinarizer()
            y = lb.fit_transform(labels)


            #split data into training and test set
            X_train, X_test, y_train, y_test = train_test_split(X, 
                                                                y, shuffle=True,
                                                                test_size=0.2, 
                                                                random_state=100)



            #create clients -- Horizontal FL
            clients = create_clients(X_train, y_train, num_clients=number_of_clients, initial='client')

            #process and batch the training data for each client
            clients_batched = dict()
            for (client_name, data) in clients.items():
                clients_batched[client_name] = batch_data(data)


                #process and batch the test set  
            test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

            #==============================================
            # Traditional FedAvg 2017
            #==============================================
            #-----------------------------------------------


            all_results=list()

            #create optimizer
            lr = 0.01 
            loss='binary_crossentropy'
            metrics = ['accuracy']
            optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=lr, 
                            decay=lr / comms_round, 
                            momentum=0.9
                           )

            #initialize global model
            smlp_global = SimpleMLP()
            global_model = smlp_global.build(X.shape[1],1)
            #-----------------------------------------------


            print('|=======================|')
            print('|Traditional FedAvg 2017|')
            print('|=======================|')

            #commence global training loop
            for comm_round in range(comms_round):

                # get the global model's weights - will serve as the initial weights for all local models
                global_weights = global_model.get_weights()

                #initial list to collect local model weights after scalling
                scaled_local_weight_list = list()

                #randomize client data - using keys
                client_names= list(clients_batched.keys())
                random.shuffle(client_names)

                #loop through each client and create new local model
                for client in client_names:
                    smlp_local = SimpleMLP()
                    local_model = smlp_local.build(X.shape[1],1)
                    local_model.compile(loss=loss, 
                                  optimizer=optimizer, 
                                  metrics=metrics)

                    #set local model weight to the weight of the global model
                    local_model.set_weights(global_weights)

                    #fit local model with client's data
                    local_model.fit(clients_batched[client], epochs=32, verbose=0)

                    #scale the model weights and add to list
                    scaling_factor = weight_scalling_factor(clients_batched, client)
                    scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
                    scaled_local_weight_list.append(scaled_weights)

                    #clear session to free memory after each communication round
                    keras.backend.clear_session()

                #to get the average over all the local model, we simply take the sum of the scaled weights
                average_weights = sum_scaled_weights(scaled_local_weight_list)

                #update global model 
                global_model.set_weights(average_weights)

                #test global model and print out metrics after each communications round
                for(X_test, Y_test) in test_batched:
                    global_acc, global_loss, global_f1, global_precision, global_recall, global_auc, global_fpr = test_model(X_test, Y_test, global_model, comm_round)
                    all_results.append([global_acc,global_loss.numpy(),global_f1, global_precision, global_recall, global_auc, global_fpr])



            all_R = pd.DataFrame(all_results, columns=['global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
            flname = 'results/round-'+str(r)+'/'+str(cl)+'-clients/FedAvg-'+dataset[d]+'-results.csv'
            all_R.to_csv(flname, index=None)
            
            
            all_avg.append(np.concatenate(([dataset[d],r,cl],np.mean(all_results,axis=0)))) #Storing avg values for each dataset
            all_std.append(np.concatenate(([dataset[d],r,cl],np.std(all_results,axis=0)))) #Storing std values sfor each dataset
            

            

            
ALL_AVG = pd.DataFrame(all_avg, columns = ['Dataset', 'num of round', 'num of cliends','global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv(f'FedAvg-results.csv')     

ALL_STD = pd.DataFrame(all_std, columns = ['Dataset', 'num of round', 'num of cliends','global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')

In [ ]:
ALL_AVG = pd.DataFrame(all_avg, columns = ['Dataset','Rounds','no-clients','global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_AVG.to_csv('FedAvg-all-avg-results.csv')

ALL_STD = pd.DataFrame(all_std, columns = ['Dataset','Rounds','no-clients','global_acc', 'global_loss', 'global_f1', 'global_precision', 'global_recall', 'global_auc', 'global_fpr'])
ALL_STD.to_csv('FedAvg-all-std-results.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1)
# make a little extra space between the subplots
fig.subplots_adjust(hspace=0.5)

s1 = np.array(all_results) #FedAvg

t = range(0,s1.shape[0])

ax1.plot(t, s1[:,0],label='Acc of FedAvg')
ax1.set_xlim(0,s1.shape[0])
ax1.set_xlabel('Rounds')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0.98,1)
ax1.grid(True)
ax1.legend()


ax2.plot(t, s1[:,1],label='Error of FedAvg')
ax2.set_xlim(0, s1.shape[0])
ax2.set_xlabel('Rounds')
ax2.set_ylabel('error')
ax2.grid(True)
ax2.legend()

plt.show()



In [ ]:
s1.shape